# Channel capacity from the HighMI_13 data set
 
This is a dataset from an experiment designed specifically to estimate the number of antigen categories encoded by the cytokine latent space. It contains 9 experimental replicates of the cytokine time series for 8 OVA peptides (N4 down to E1) and each of 4 concentrations (1 $\mu$M down to 1 nM). 

### Another explanation of channel capacity
What we want to compute is channel capacity between antigen quality $Q$ (quantified by the EC$_{50}$) and latent space time trajectories, characterized by the values of model parameters fitted on them. Specifically, we take the vector, continuous random variable $\mathbf{X} = (a_0, t_0, \theta)$, made of three parameters from the force model with matching, as the description of a trajectory in latent space in response to some antigen $Q$. The mutual information $MI(\mathbf{X}; Q)$ measures how much information about antigen quality $Q$ we gain by knowing the value of the three model parameters in $\mathbf{X}$ that describe (in latent space) the resulting cytokine time series. The MI is defined as

$$ MI(\mathbf{X}; Q) = \sum_q p_Q(q) \int_{\mathcal{X}} \mathrm{d}\mathbf{x} f_{\mathbf{X}|Q=q}(\mathbf{x}) \log_2{\left( \frac{f_{\mathbf{X}|Q=q}(\mathbf{x})}{f_{\mathbf{X}}(\mathbf{x})} \right)}$$

where $\mathcal{X}$ is the range of possible values of the random variable $\mathbf{X}$, $p_Q(q)$ is the probability to have an antigen of quality $Q$, $f_{\mathbf{X}|Q=q}$ is the conditional probability density of $\mathbf{X}$ in response to a given antigen quality $Q$, and $f_{\mathbf{X}}$ is the marginal probability density of $\mathbf{X}$, given by $\sum_q p_Q(q) f_{\mathbf{X|Q=q}}(\mathbf{x})$. The conditional probability density can be though of as an input-output function that maps a given antigen quality to a distribution of possible latent space trajectories. In other words, it characterizes the latent space as an information channel that encodes antigen quality (hence the notion of antigen encoding). 

The antigen probability distribution $p_Q$ is somewhat arbitrary: is it the frequency of a given antigen in the experiment? Is it the abundance of antigens of that strength $\textit{in vivo}$? We can actually characterize the information content of the cytokine latent space independently from $p_Q$ by computing instead channel capacity, $C(\mathbf{X}; Q)$. This quantity gives the most information one can possibly extract from the channel, whatever $p_Q$ is. It is commonly used for this purpose in information theory.  Mathematically, 

$$ C(\mathbf{X}; Q) = \max_{p_Q} MI(\mathbf{X}; Q) $$

In other words, we maximize mutual information over the space of all possible input (antigen) distributions. Hence, $C(\mathbf{X}; Q)$ no longer depends on $p_Q$. 


## Steps to compute channel capacity
1. Process cytokine data: smoothing, computation of integrals, 
2. Projection of cytokine trajectories to latent space and (min max scaling) normalization with training data normalization parameters (min and max of each cytokine in the training data). 
3. Fitting model parameters on each time series in latent space: gives $\mathbf{x}$ samples
4. Fitting multivariate normal distributions on the distributions of model parameters for each antigen (i.e., the distributions $f_{\mathbf{X}|Q=q}$)
5. Interpolation, as a function of $Q$, of the means and covariance matrices of the multivariate normal distributions. 
6. Calculation of the channel capacity between $Q$ and $\mathbf{X}$ with the Blahut-Arimoto and Monte Carlo integrations for the multidimensional integrals over the domain of $\mathbf{X}$. 


In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from time import perf_counter
import os

from ltspcyt.scripts.process_raw_data import process_file, process_file_filter
from ltspcyt.scripts.sigmoid_ballistic import return_param_and_fitted_latentspace_dfs, compute_v2_v1
from utils.statistics import build_symmetric
from utils.distrib_interpolation import (eval_interpolated_means_covs, interpolate_params_vs_logec50, 
                                         stats_per_levels, compute_cholesky_dataframe)
import utils.custom_pandas as custom_pd
from utils.extra_pairplots import dual_pairplot

In [ ]:
%matplotlib inline

# 1. Process raw data, filter out noise
The processing steps we needed for this dataset:
1. Removing some outlier time series caused by stronger peptides splashing from one well to another during preparation of these plates for the robot. 
2. Filtering of cytokine trajectories that are purely noise, using the same filtering method based on the Kolmogorov-Smirnov test that was applied to the human TCR dataset (see supplementary information). 
3. Usual moving average and spline smoothing in log scale (supp. section 2)

### 1.1 Remove known outlier trajectories

Due to a shortage of tips, plate inversion was used to remove supernatant during preparation. This caused stronger peptides to splash in some wells, which caused unnatural high cytokine time series in those wells. 

Look at the raw data plot below (IFN-$\gamma$ shown, but other cytokines display the same effect). 
- Replicates 2, 5, 8 of V4 at 1 nM and E1 at 1 $\mu$M were contaminated with stronger peptides. This shows up as unrealistically high cytokine responses. These replicates are in homologous positions in each of the 3 plates (plate 1 is reps 1-3, plate 2 is reps 4-6, plate 3 is reps 7-9, and hence 2,5,8 correspond to the second replicate in all 3 plates). 
- Similarly, replicates 3-6-9 of G4 at 1 nM have been contaminated, as well as replicates 3-6-9 of E1 at 1 $\mu$M.

The faulty V4, G4 replicates can just be dropped, but the E1 replicates have to be replaced, because E1 is the null response reference for further noise filtering (processing step 2). Since correct E1 1$\mu$M trajectories are just uncorrelated noise, we replace the faulty ones this by randomly sampling points from the three non-faulty replicates to replace the fault replicates. 

In [ ]:
# Import raw cytokine dataset
df_raw = pd.read_pickle(os.path.join("data", "final", 
            "cytokineConcentrationPickleFile-20210619-HighMI_13-final.pkl"))

In [ ]:
# Plot of all IFNg time series (for all peptides [columns] at all concentrations [rows] 
# and all replicates [different line colors])
peptide_order = ["N4", "Q4", "T4", "V4", "G4", "E1", "A2", "Y3"]
conc_order = ["1uM", "100nM", "10nM", "1nM"]
g = sns.relplot(data=np.log10(df_raw.unstack("Cytokine").stack("Time")).reset_index(), 
           x="Time", y="IFNg", col="Peptide", row="Concentration", hue="Replicate", palette="Set2", 
           kind="line", height=1.3, lw=2., col_order=peptide_order, row_order=conc_order)

for i in range(g.axes.shape[0]):
    for j in range(g.axes.shape[1]):
        g.axes[i, j].set_title(peptide_order[j] + " " + conc_order[i])

plt.show()
plt.close()

In [ ]:
# Function to replace faulty E1 replicates with points randomly sampled from correct remaining replicates
# Only use on time series which are pure experimental noise (i.e. no true cytokine time series)
def replace_time_series(df, lbl_to_replace, good_replicates, rndgen):
    """ The df should be indexed (Cytokine, Replicate, TCellNumber, Peptide, Concentration). 
    Time should be in the columns. 
    lbl_to_replace specifies slice(None), Replicate, ..., Concentration
    Warning: modifies the dataframe in place
    """
    choices = rndgen.choice(good_replicates, size=len(df.columns), replace=True)
    for i, tim in enumerate(df.columns):
        df.loc[lbl_to_replace, tim] = df.loc[(slice(None), choices[i], *lbl_to_replace[2:]), tim].values
    return 0

In [ ]:
df_raw = pd.read_pickle(os.path.join("data", "final", 
            "cytokineConcentrationPickleFile-20210619-HighMI_13-final.pkl"))
randomgen = np.random.default_rng(seed=1349839)
for rep in ["2", "3", "5", "6", "8", "9"]:
    replace_time_series(df_raw, (slice(None), rep, "30k", "E1", "1uM"), ["1", "4", "7"], randomgen)

# Drop faulty replicates for other peptides
for cyto in df_raw.index.get_level_values("Cytokine").unique():
    for rep in ["2", "5", "8"]:
        df_raw = df_raw.drop((cyto, rep, "30k", "V4", "1nM"), axis=0)
    for rep in ["3", "6", "9"]:
        df_raw = df_raw.drop((cyto, rep, "30k", "G4", "1nM"), axis=0)

### Write back to a file
fname_corrected = "cytokineConcentrationPickleFile-20210619-HighMI_13_corrected-final.pkl"
df_raw.to_pickle(os.path.join("data", "final", fname_corrected))

### 1.2 and 1.3 Filter noise, smooth and fit splines the corrected raw data
Here, E1 is the reference for a null response. 
Trajectories for E1 and some other non-responding conditions have large fluctuations in the 100 fM to 1 pM cytokine concentration range. These concentrations are in the low end of the calibration curve and are thus amplified in variation. We filter them out, after first log-transforming the raw data. 

We compare the statistics of each IFN-$\gamma$ time series to the corresponding E1 trajectories (same replicate, same peptide concentration) with the Kolmogorov-Smirnov test, and if they are found to be similar, IL-2, IL-17A, TNF and IL-6 are set to zero for both trajectories. E1 trajectories have those cytokines set to zero as well, because we know they are experimental background noise. IFN-$\gamma$ is not set to zero, because we want to keep some signal, otherwise the trajectories are perfectly zero in latent space, which is somewhat artificial. 

Then, the usual processing is applied on the corrected data: finding and fixing missing time points, moving average smoothing, cubic B-spline fitting. 

In [ ]:
# Filtering out null-like cytokine time series. 
_, _, _, df_wt = process_file_filter(folder=os.path.join("data", "final"), file=fname_corrected,
                        do_filter_null=True, null_reference="E1", filter_pval=0.5, choice_filter_cyto="IFNg", 
                        choice_remove_cyto=["IL-2", "IL-17A", "TNFa", "IL-6"], remove_il17=False,
                        split_filter_levels=["Replicate", "Concentration"], do_self_filter=True)

# Write the corrected and processed file to hdf
#df_wt.to_hdf(os.path.join("data", "final", "HighMI_13_corrected.hdf", key="df")

# 2. Project to latent space and normalize

In [ ]:
peptides = ["N4", "Q4", "T4", "V4", "G4", "E1", "A2", "Y3"]
concentrations = ["1uM", "100nM", "10nM", "1nM"]
times = np.arange(1, 73)
tcellnum = "30k"
tcn_fit = "30k"
folder = "highmi13"
suffix = "_HighMI_13"


df_min, df_max = pd.read_pickle(os.path.join("data", "trained-networks", "min_max-thomasRecommendedTraining.pkl"))
mlpcoefs = np.load(os.path.join("data", "trained-networks", "mlp_input_weights-thomasRecommendedTraining.npy"))

cytokines = df_min.index.get_level_values("Cytokine")
df_wt = pd.concat({"HighMI_13":df_wt}, names=["Data"])

# Normalize and projection to latent space
df = df_wt.unstack("Time").loc[:, ("integral", cytokines, times)].stack("Time")
df = (df - df_min)/(df_max - df_min)
df_proj_exp = pd.DataFrame(np.dot(df, mlpcoefs), index=df.index, columns=["Node 1", "Node 2"])

# 3. Fit a model on all selected integral time courses
We use the force model with matching. 

We need a special dictionary of parameter bounds, because in this experiment, some theta values are below $-\pi/2$, which is not allowed in the code by default. 

Moreover, since the IL-2 consumption is slower in this dataset, we don't quite reach the final dynamical phase with constant slope. Hence, we can't fit properly that final slope; instead, we import the final slope of another dataset where IL-2 consumption was faster. 


#### Remark
Instead of processing HighMI_13 and fitting a model on it, you could bypass steps 1-3 by importing a dataframe of fitter model parameter values, such as those saved by the `fit_latentspace_model.ipynb`notebook. 

In [ ]:
# Process HighMI_1 to get a v2/v1 slope
df_highmi1 = pd.concat({"HighMI_1-{}".format(i): pd.read_hdf("data/processed/HighMI_1-{}.hdf".format(i)) 
                        for i in range(1, 5)}, names=["Data"])
df_highmi1 = df_highmi1.unstack("Time").loc[:, ("integral", cytokines, times)].stack("Time")
df_highmi1 = (df_highmi1 - df_min)/(df_max - df_min)
df_proj_highmi1 = pd.DataFrame(np.dot(df_highmi1, mlpcoefs), index=df_highmi1.index, columns=["Node 1", "Node 2"])
v2v1_mean_slope = compute_v2_v1(df_proj_highmi1, slope_type="mean", reject_neg_slope=True)

### Fit model parameters
We fit peptides N4 to T4 and V4 to E1 with different regularization terms. The default regularization coefficients (used for N4, ..., T4) are too large for trajectories that remain close to the origin; they cause too many trajectories to be fitted with zero parameter values. 

Moreover, we include a small regularization term that enforces our prior knowledge of a correlation between $a_0$, $t_0$, and $\theta$. The linear correlation enforced was determined from a prior fit without regularization terms for correlation. 

In [ ]:
### Fitting parameters for N4 down to T4
pep_selection = ["N4", "Q4", "T4", "A2", "Y3"]

fit_vars={"Constant velocity":["v0","t0","theta","vt"],"Constant force":["F","t0","theta","vt"],
         "Sigmoid":["a0", "t0", "theta", "v1", "gamma"], 
         "Sigmoid_freealpha":["a0", "t0", "theta", "v1", "alpha", "beta"]}

time_scale = 20.0
duration = np.amax(times)

# Special parameter boundaries for this dataset. Model "Constant force" is not used anymore
# but included for backward compatibility with the code. 
special_bounds_dict = {
    'Constant velocity':[(0, 0, -np.pi/3, 0), (6/20*time_scale, 5/20*time_scale, np.pi, 5/20*time_scale)],
    'Constant force': [(0, 0, -np.pi/3, 0), (6/20*time_scale, 5/20*time_scale, np.pi, 5/20*time_scale)],
    'Sigmoid':[(0, 0, -2*np.pi/3, -2/20*time_scale, time_scale/100),
                (8/20*time_scale, (duration + 100)/time_scale, np.pi/3, 2/20*time_scale, time_scale/2)],
    'Sigmoid_freealpha':[(0, 0, -2*np.pi/3, 0, time_scale/72, time_scale/72),
                        (8/20*time_scale, (duration + 100)/time_scale, np.pi/3, 2/20*time_scale,
                        time_scale/5, time_scale/2)]
}

# Fitting
choice_model = "Sigmoid_freealpha"
nparameters = len(fit_vars[choice_model])

# Regularization
regul_rate = 0.9*np.ones(len(fit_vars[choice_model]))
regul_rate[1] = 0.7  # reduce regularization on t0 because 0.9 forces too many trajectories to zero

# Each row specifies [i, j, a, b, c] for a term of the form c*(p[i] - a*p[j] - b)^2 in the cost function 
# The array must contain ints (to slice with i, j), so we input floats as numerators over 10,000
# Removing this regularization does not significantly change the parameter space. 
params_correls = np.asarray([
    [1, 0, int(1.9*10000), 0, int(0.05*10000)], # t0 = 1.9*a0
    [2, 0, int(0.75*10000), int(-np.pi/2*10000), int(0.1*10000)]  # theta = 0.75*a0 - pi/2
], dtype=int).T

start_time = perf_counter()

ret = return_param_and_fitted_latentspace_dfs(
            df_proj_exp.loc[df_proj_exp.index.isin(pep_selection, level="Peptide")], 
            choice_model, time_scale=time_scale, reg_rate=regul_rate, reject_neg_slope=v2v1_mean_slope, 
            special_bounds_dict=special_bounds_dict, correls=params_correls)
df_params1, df_compare1, df_hess1, df_v2v11 = ret

end_t = perf_counter()
print("Time to fit: ", end_t - start_time, "s")
del start_time, end_t

In [ ]:
### Fitting parameters for V4 to E1
pep_selection = ["V4", "G4", "E1"]

time_scale = 20.0
duration = np.amax(times)

# Regularization: slightl lower for the weaker peptides
regul_rate = 0.6*np.ones(len(fit_vars[choice_model]))
regul_rate[1] = 0.4

# Lightly enforcing parameter correlations
params_correls = np.asarray([
    [1, 0, int(1.8*10000), 0, int(0.1*10000)], # t0 = 1.9*a0
    [2, 0, int(0.5*10000), int(10000*-np.pi/2), int(0.1*10000)]  # theta = 0.5*a0 - 2
], dtype=int).T

start_time = perf_counter()

# Use a different final slope, because for those peptides we can properly fit it
ret2 = return_param_and_fitted_latentspace_dfs(
            df_proj_exp.loc[df_proj_exp.index.isin(pep_selection, level="Peptide")], 
            choice_model, time_scale=time_scale, reg_rate=regul_rate, reject_neg_slope=v2v1_mean_slope, 
            special_bounds_dict=special_bounds_dict, correls=params_correls)
df_params2, df_compare2, df_hess2, df_v2v12 = ret2

end_t = perf_counter()
print("Time to fit: ", end_t - start_time, "s")
del start_time, end_t

In [ ]:
# Combine the fit results for the two previous sets of peptides
df_params = df_params1.append(df_params2)
df_compare = df_compare1.append(df_compare2)
df_hess = df_hess1.append(df_hess2)
df_v2v1 = df_v2v11.append(df_v2v12)

# We don't need the individual dataframes anymore
del df_params1, df_compare1, df_hess1, df_v2v11, df_params2, df_compare2, df_hess2, df_v2v12

### Plotting the result of the fits

In [ ]:
if tcn_fit == "all":
        df_params_sel = df_params
else:  
    df_params_sel = df_params.xs(tcellnum, level="TCellNumber", axis=0)
hue_ord = [a for a in peptides]
h = sns.pairplot(data=df_params_sel.reset_index().query("Peptide in @hue_ord"), 
                 vars=fit_vars[choice_model][:3], hue="Peptide", hue_order=hue_ord, 
                 plot_kws={"s":16, "alpha":0.8}, diag_kind="none")
legend = h.legend

#h.fig.savefig(os.path.join("figures", folder, "pairplot_{}_HighMI_13.pdf".format(choice_model)), 
#    transparent=True, bbox_extra_artists=(legend,), bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
data = df_compare.loc[("HighMI_13", slice(None), "30k", peptides, slice(None), slice(None), slice(None), "integral"), :]
h=sns.relplot(data=data.reset_index(), x="Node 1",y="Node 2", kind="line", sort=False,
            hue="Peptide", hue_order=peptides,
            size="Concentration", size_order=concentrations,
            style="Replicate", col="Processing type")
legend = h.legend
h.fig.axes[0].set_aspect("equal")
h.fig.axes[1].set_aspect("equal")
#h.fig.savefig(os.path.join("figures", folder, "latentspace_comparison_HighMI13.pdf"), 
#    format="pdf", bbox_inches="tight")
plt.show()
plt.close()

#### Removing a few more outliers
Some parameter values for E1, G4 and V4 do not make sense, for instance the very high $\theta$ values that are completely off the cloud of points. 

In [ ]:
# Those G4, V4, E1 fits don't represent the difference with other peptides and are just noise
df_params2 = df_params.loc[~(df_params.index.isin(["E1"], level="Peptide")*(df_params["theta"] > -np.pi/3))]
df_params2 = df_params2.loc[~(df_params2.index.isin(["T4"], level="Peptide")*(df_params2["theta"] < -np.pi/2))]
df_params2 = df_params2.loc[~(df_params2.index.isin(["G4"], level="Peptide")*(df_params2["theta"] > -np.pi/3))]

In [ ]:
# Uncomment to save the parameter fits on disk
#df_params2.to_hdf(os.path.join("results", "fits", "df_params_{}_{}.hdf".format(
#    choice_model.replace(" ", "_"), "HighMI_13")), key="df_params")

# 4. Fit multivariate normal distributions in model parameter space 
 

In [ ]:
params_to_keep = ["a0", "t0", "theta"]
levels_group = ["Peptide"]

# THIS IS WHERE THE GAUSSIANS ARE ACTUALLY FITTED
# Use df_params to mix all T cell numbers, use df_params_sel to use only 100k
if tcn_fit == "all":
    ret = stats_per_levels(df_params2, levels_groupby=levels_group, feats_keep=params_to_keep)
else:
    ret = stats_per_levels(df_params2.xs(tcn_fit, level="TCellNumber", axis=0), 
                           levels_groupby=levels_group, feats_keep=params_to_keep)
df_params_means, df_params_means_estim_vari, df_params_covs, df_params_covs_estim_vari, ser_npts = ret
df_params_covs_estim_vari = np.abs(df_params_covs_estim_vari)

## Check that the results make some sense by sampling the generated gaussians
Create a synthetic parameter space, basically. I am sure it will look very different! 

In [ ]:
nsamples = 40
seed = 1357642
rnd_gen = np.random.default_rng(seed=seed)
if len(levels_group) == 1:
    new_index = pd.MultiIndex.from_product([df_params_means.index] + [range(nsamples)], 
                                      names=[df_params_means.index.name, "Sample"])
else:
    new_index = pd.MultiIndex.from_product([*zip(*df_params_means.index)] + [range(nsamples)], 
                                      names=[df_params_means.index.names] + ["Sample"])
df_params_synth = pd.DataFrame(index=new_index, columns=params_to_keep, dtype=np.float64)
df_params_synth.columns.name = "Parameter"

# Sample from the fitted gaussians
for key in df_params_means.index:
    cov_mat = build_symmetric(df_params_covs.loc[key].values)
    mean_vec = df_params_means.loc[key].values
    df_params_synth.loc[key] = rnd_gen.multivariate_normal(mean_vec, cov_mat, nsamples)
    
# Concatenate the data and synthetic sample points, for plotting purposes   
if tcn_fit == "all":
    params_remove = list(set(df_params2.index.names).difference(levels_group))
    df_params_both = df_params2.droplevel(params_remove).sort_index()
else:
    params_remove = list(set(df_params2.index.names).difference(levels_group))
    params_remove.remove("TCellNumber")
    df_params_both = df_params2.xs(tcn_fit, level="TCellNumber", axis=0).droplevel(params_remove).sort_index()

df_params_both = df_params_both.loc[:, params_to_keep[0]:params_to_keep[-1]]
print(df_params_both.groupby(levels_group).count().values)

idx = np.concatenate([np.arange(n) for n in df_params_both.groupby(levels_group).count().sort_index().values[:, 0]])
df_params_both["Sample"] = idx
df_params_both = df_params_both.set_index("Sample", append=True)
df_params_both = pd.concat([df_params_both, df_params_synth], axis=1, keys=["Data", "Synth"], names=["Source", "Parameter"])
df_params_both = df_params_both.stack("Source")
print(df_params_both)

### Plot the synthetic versus real points

In [ ]:
peptides = ["N4", "A2", "Y3", "Q4", "T4", "V4", "G4", "E1"]
pep_color_order = ["N4", "Q4", "T4", "V4", "G4", "E1", "A2", "Y3", "A8", "Q7"]
pep_palette = {pep_color_order[i]:sns.color_palette()[i] for i in range(len(pep_color_order))}
palette_order = [pep_palette.get(a) for a in peptides]
rename_dict = {"a0":r"$a_0$", "t0":r"$t_0$", "theta":r"$\theta$"}
params_to_keep2 = [rename_dict.get(a, a) for a in params_to_keep]
df_params_plot = custom_pd.xs_slice(df_params_both.rename(rename_dict, axis=1, level="Parameter"), 
                                    name="Peptide", lvl_slice=peptides, axis=0).reset_index()

fig, axes, legend = dual_pairplot(data=df_params_plot, vari=params_to_keep2, dual_lvl="Source", 
              dual_labels=["Data", "Synthetic"], hue="Peptide",
              dual_hues = [(0.5, 0.5, 0.5), plt.cm.viridis([206])[0]], palette=palette_order, 
              hue_order=peptides, alpha=0.8, s=16, edgecolors=None)

# Clean up layout
fig.set_size_inches(6.5, 6.5)
fig.tight_layout(h_pad=0.5, w_pad=0.5)


#fig.savefig(os.path.join(
#   "figures", folder, "pairplot_synthreal_dual_{}_HighMI_13.pdf".format(choice_model.replace(" ", "_"))), 
#   transparent=True, bbox_extra_artists=(legend,), bbox_inches='tight')

In [ ]:
# Overlay data and synthetic sample points
h = sns.pairplot(
    data=custom_pd.xs_slice(df_params_both, name="Peptide", 
                            lvl_slice=peptides, axis=0).reset_index(), 
    vars=params_to_keep, hue="Source", hue_order=["Synth", "Data"], palette=plt.cm.viridis([40, 206]),
    plot_kws={"s": 16, "alpha":0.7})
legend = h.legend

#h.fig.savefig(os.path.join(
#    "figures", folder, "pairplot_synthreal_{}_reg10_moredata_TCN{}.pdf".format(choice_model, tcn_fit)), 
#    transparent=True, bbox_extra_artists=(legend,), bbox_inches='tight')
plt.show()
plt.close()

## Compute the Cholesky decomposition of the covariance matrices
To preserve positive definiteness, we will have to interpolate the elements of the covariance matrix's Cholesky decomposition, $\Sigma = L L^T$. Then, once we have $L(\mathrm{EC}_{50})$ interpolated as a function of EC$_{50}$, we can ensure positive definiteness by constructing $\Sigma(\mathrm{EC}_{50}) = L(\mathrm{EC}_{50}) L^T(\mathrm{EC}_{50})$, which is then necessarily symmetric and positive definite because it has a Cholesky decomposition with non-negative diagonal elements. 

### Variance of the Cholesky decomposition estimator
To interpolate elements of the Cholesky matrices, we need error bars on the statistical estimates of those elements. The paper Olkin, 1985, *Estimating a Cholesky Decomposition* studies a statistical estimator of the Cholesky decomposition of the covariance matrix, and the variance of that estimator. 

Olkin gives an unbiased estimator for $\Psi$, which is surprisingly not just the Cholesky decomposition of the covariance matrix estimator. The problem with Olkin's estimator is that (because it is unbiased for $\Psi$ itself) it does not reconstruct an unbiased covariance matrix, i.e. $\langle \hat{\Psi} \hat{\Psi}^T \rangle \neq \Sigma$. Because I only care about $\Sigma$ in the end, I just define instead $\hat{\Psi}$ from $\hat{\Psi} \hat{\Psi}^T = \hat{\Sigma}$, where $\hat{\Sigma}$ is the covariance matrix estimator. Both estimators are convergent anyways as $K \rightarrow \infty$ ($K = $ number of sample points). 

Olkin's paper gives a correct derivation of the variance of its unbiased estimator of $\Psi$, which can readily be applied to my biased estimator (they are related by a scaling factor). However, Olkin does not report the final result for that variance properly. Specifically the paper proves that the $u_{ij}^2$ variables follow a $\chi^2$ distribution, and hence the variance of the $u_{ij}$ is related to the *mean* of the $\chi^2$ distribution for $u^2$. However, Olkin uses the variance of the $chi^2$ distribution, which leads to variances of the estimator that do not decay to zero as the number of points $K \rightarrow \infty$. 

In any case, following the derivation carefully and correcting the mistake in the paper's final result,the variance of the Cholesky matrix estimator is (replacing the unknown true $\Psi$ by $\hat{\Psi}$):

$$ \mathrm{Var}[\hat{\Psi}^{ii}] =  \frac{K-i - a_i^2}{K-1} (\Psi^{ii})^2 $$

$$	\mathrm{Var}[\hat{\Psi}^{ij}] = \frac{1}{K-1} \sum_{l=j+1}^i (\Psi^{il})^2 + \frac{K - j - a_j^2}{K-1} \Psi^{ij} \quad (i \neq j) $$
where 
$$ a_i = \sqrt{2} \frac{\Gamma((K-i+1)/2)}{\Gamma((K-i)/2)} \, . $$

The variances on the estimator matrix elements go to zero as $K \rightarrow \infty$ due to the asymptotic behaviour of the $\Gamma$ function, which implies $a_j^2 \sim (K - j) - \frac{1}{2}$. 

Therefore, all we need to estimate $L$ and the variance of that estimator is the covariance matrix estimator, $\hat{\Sigma}$, which we Cholesky-decompose to get our (biased) $\hat{L}$, and the number of sample points $N$, because the variance of the estimator (which is biased, not exact like our long formula for the variance of $\hat{\Sigma}$ itself) only depends on elements of $\hat{L}$ and $n$. This is highly convenient. 

In [ ]:
## See the implementation of the estimator in utils.statistics
df_params_chol, df_params_chol_estim_vari = compute_cholesky_dataframe(df_params_covs, ser_npts)
print(df_params_chol_estim_vari)

### Save fitted multivariate normal parameter distributions
Convert the cell below to code to save. 

# 5. Interpolate multivariate normal distributions as a function of EC$_{50}$
For each mean vector and covariance Cholesky decomposition matrix element, use a smoothing spline interpolation to get the value of those elements at arbitrary $\log_{10}{\mathrm{EC}_{50}}$. 

For the interpolation of a given distribution parameter $p$ as a function of $g = \log_{10}{\mathrm{EC}_{50}}$, we use this procedure:
1. Use the $p(g_i)$ values for each $g_i$ corresponding to one of the peptides
2. For error bars, use the square root of the statistical variance on this estimated parameter, $\sqrt{\sigma_p^2(g_i)}$. 
3. Fit a smoothing UnivariateSpline (Dierckx 1975) on $(g_i, p(g_i))$ with weighting factors $1/\sqrt{\sigma_p^2(g_i)}$ and the smoothing factor set to $0.25$ times the default, that is, $0.25$ times the number of points $g_i$. 
4. Evaluate the smoothing spline at the $g_i$ to get $\hat{p}(g_i)$. The spline may have large non-monotonous artifacts between the data points, but it should be as close to the points as the smoothing allows, so any potential large artifact should not be significant at those locations. 
5. Fit a PCHIP monotonous cubic interpolator through those values, $(g_i, \hat{p}(g_i))$. 


In [ ]:
df_ec50s_refs = pd.read_json(os.path.join("data", "misc", "potencies_df_2021.json"))
df_ec50s_refs.columns.name = "Reference"; df_ec50s_refs.index.name = "Peptide"
print(df_ec50s_refs)
ser_ec50s_avglog = np.log10(df_ec50s_refs).mean(axis=1)

In [ ]:
# Add error bars; can't use relplot or catplot anymore, 
# they don't have the option to include known standard deviations
def plot_params_vs_logec50(df_estim, df_estim_vari, ser_x, cols_plot=None, 
                        ser_interp=None, x_name="Peptide", col_wrap=3):
    """ Optional df_interp, which contains interpolating splines for each parameter
    as a function of the x variable (log EC50, usually). 
    """
    if cols_plot is None:
        nplots = len(df_estim.columns)
        cols_plot = df_estim.columns
    else:
        nplots = len(cols_plot)
    
    nrows = nplots // col_wrap + min(1, nplots % col_wrap)  # Add 1 if there is a remainder. 
    ncols = min(nplots, col_wrap)
    
    fig, axes = plt.subplots(nrows, ncols, sharex=True, sharey=False)
    fig.set_size_inches(3*ncols, 2.5*nrows)
    axes = axes.flatten()
    
    for i in range(nplots):
        estim = df_estim[cols_plot[i]]
        stds = np.sqrt(df_estim_vari[cols_plot[i]])
        x_labels = estim.index.get_level_values(x_name)
        xpoints = ser_x.reindex(x_labels)  # assume ser_x has a single index level?
        xmin, xmax = np.amin(xpoints), np.amax(xpoints)
        axes[i].errorbar(xpoints, estim, yerr=stds, ls='none', marker="o", ms=5, label="Fit")
        axes[i].set_ylabel(cols_plot[i])
        xr = xmax - xmin
        yr = np.amax(estim) - np.amin(estim)
        for j in range(len(x_labels)):
            axes[i].annotate(x_labels[j], xy=(xpoints[j]+0.01*xr, estim[j]+0.02*yr), fontsize=8)
        if ser_interp is not None:
            spl = ser_interp[cols_plot[i]]
            xrange = np.linspace(xmin, xmax, 201)
            axes[i].plot(xrange, spl(xrange), lw=1.5, label="Interpolation")
            axes[i].legend()
    return [fig, axes]

## Interpolate

In [ ]:
ser_splines_means = interpolate_params_vs_logec50(df_params_means, df_params_means_estim_vari, 
                                      ser_ec50s_avglog, x_name="Peptide")

# Plot the interpolation versus the data
fig, axes = plot_params_vs_logec50(df_params_means, df_params_means_estim_vari, ser_ec50s_avglog, 
                             ser_interp=ser_splines_means, cols_plot=None, x_name="Peptide", col_wrap=3)
for ax in axes[-3:]:  # 3 is col_wrap
    ax.set_xlabel(r"$\log_{10}{\mathrm{EC}_{50}}$ [-]")
fig.tight_layout()
#fig.savefig(os.path.join("figures", folder, "mean_vs_logec50_{}_{}.pdf".format(
#     choice_model.replace(" ", "_"), suffix)), transparent=True)
plt.show()
plt.close()

In [ ]:
ser_splines_covs = interpolate_params_vs_logec50(df_params_covs, df_params_covs_estim_vari, 
                                      ser_ec50s_avglog, x_name="Peptide")

ser_splines_chol = interpolate_params_vs_logec50(df_params_chol, df_params_chol_estim_vari, 
                                      ser_ec50s_avglog, x_name="Peptide")
fig, axes = plot_params_vs_logec50(df_params_chol, df_params_chol_estim_vari, ser_ec50s_avglog, 
                             ser_interp=ser_splines_chol, cols_plot=None, x_name="Peptide", col_wrap=3)
for ax in axes[-3:]:  # 3 is col_wrap
    ax.set_xlabel(r"$\log_{10}{\mathrm{EC}_{50}}$ [-]")
fig.tight_layout()
#fig.savefig(os.path.join("figures", folder, "cholesky_vs_logec50_{}_{}.pdf".format(
#     choice_model.replace(" ", "_"), suffix)), transparent=True)
plt.show()
plt.close()

## Discretize the EC50 axis
Sampling closely spaced but still discrete $Q$ values allows us to use the Blahut-Arimoto algorithm to optimize over a discrete $p_Q$. The only different is that integrals over $\mathcal{X}$ (for each value of $Q$) are now continuous; we perform them with Monte Carlo integration in our code for better speed. 

In [ ]:
## Discretize the $log{EC_{50}}$ axis
# Let's try $n=25$, which seems enough to extract all the information available
n_inputs = 25
min_logec50 = ser_ec50s_avglog.min()
max_logec50 = ser_ec50s_avglog.max()
bins_logec50 = np.linspace(min_logec50, max_logec50, n_inputs+1)
# Take the midpoints of those bin separators
sampled_logec50 = bins_logec50[:n_inputs] + np.diff(bins_logec50)/2
print(sampled_logec50)

## Compute the means and covariance matrices at each EC50, using the interpolation
n_dims = len(df_params_means.columns)  # number of parameters
meanmats, covmats, covmats_direct = eval_interpolated_means_covs(
    ser_splines_means, ser_splines_covs, ser_splines_chol, sampled_logec50, 
    n_inputs, n_dims, epsil=1e-5
)
# Uncomment to save the interpolated channel to the results/ folder
#np.save(os.path.join("results", folder, "meanmats_{}inputs{}.npy".format(n_inputs, suffix)), meanmats)
#np.save(os.path.join("results", folder, "covmats_{}inputs{}.npy".format(n_inputs, suffix)), covmats)

# 6. Compute channel capacity with Blahut-Arimoto

In [ ]:
from wurlitzer import sys_pipes
import json
from datetime import date

import chancapmc.chancapmc as chancapmc

In [ ]:
## Time and run the Blahut-Arimoto algorithm. 
seed = 42493287
reltol = 0.01

# With wurlitzer, the C code output should print here at the end of the run only
# https://github.com/minrk/wurlitzer
# To print it as it goes in the terminal running the notebook, remove the "with sys_pipes():" statement. 
start_t = perf_counter()

# Use this wurlitzer function to print C-level stdout and stderr in the notebook
with sys_pipes():
    res = chancapmc.ba_discretein_gaussout(meanmats, covmats, sampled_logec50, reltol, seed)

capacity_bits, optim_input_distrib = res
capacity_error = capacity_bits * reltol

print("Capacity = {} pm {}".format(capacity_bits, capacity_error))
print("Optimal input distribution:", optim_input_distrib)

run_duration = perf_counter() - start_t
print("Time to converge: ", run_duration, "s")

del start_t

In [ ]:
# Save all the info on the run in a JSON file
today = date.today().strftime("%d-%b-%Y").lower()
run_info = {
    "date": today, 
    "capacity_bits": capacity_bits, 
    "input_values": list(sampled_logec50.astype(float)), 
    "optim_input_distrib": list(optim_input_distrib.astype(float)),
    "n_inputs": n_inputs, 
    "run_duration (s)": run_duration, 
    "relative tolerance": reltol, 
    "absolute_error": capacity_error, 
    "params": list(set(a for lbl in ser_splines_means.index 
                          for a in lbl.split("*"))), 
    "seed": seed    
}
folder = "highmi13"
suffix = "_HighMI_13"
filename = os.path.join("results", folder, "run_log_{}ins_rtol{:.0e}{}_{}.json".format(
    n_inputs, reltol, suffix, today))

# UNCOMMENT TO SAVE: it's probably a good idea to avoid wasting the few minutes of calculation above. 
#with open(filename, "w") as hand:
#    json.dump(run_info, hand)

In [ ]:
# Make a histogram (bar plot) of the optimal input distribution
fig, ax = plt.subplots()
ax.bar(np.around(sampled_logec50, 2), optim_input_distrib, width=np.diff(sampled_logec50)[0])
ax.set_xlabel(r"$\log_{10}{(\mathrm{EC}_{50})}$ [-]", size=14)
ax.set_ylabel("Probability [-]", size=14)
ax.tick_params(which="both", labelsize=12)
ax.annotate(r"C = {:.4f} bits $\pm$ {:.2f} %".format(capacity_bits, reltol*100), xy=(0.3, 0.8), 
            xycoords="axes fraction", size=12)

# Annotate peptides
maxprob = np.amax(optim_input_distrib)
for pep in ser_ec50s_avglog.index:
    if pep not in df_params_means.index:
        continue
    ax.annotate(pep, xy=(ser_ec50s_avglog[pep], maxprob), fontsize=8, ha="center")
    ax.axvline(ser_ec50s_avglog[pep], ls=":", lw=0.5)

#fig.savefig(os.path.join("figures", folder, 
#     "optimal_logec50_distrib_{}_inputs_seed{}_rtol{:1.0e}{}.pdf".format(n_inputs, seed, reltol, suffix)))
plt.show()
plt.close()